# **Analysis of the NTU RGB+D Action Recognition Dataset**

___
___

## **1. INITIALIZATION**

### *1.1. IMPORT*

In [1]:
from IPython.display import display
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time

from torch.utils.data import Dataset

import torch

In [2]:
# to fill the requirement.txt file we use the following line of code:
import session_info
session_info.show()

In [3]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: {}".format(device))

device: cuda


### *1.2. DATA LOADING*

In [4]:
data_dir = "data/nturgbd_skeletons_cleaned/"
data_files = os.listdir(data_dir)

In [5]:
with open("data/actions.txt", 'r') as actions_file:
    actions = [line.replace('\n', '') for line in actions_file.readlines()]
    actions_file.close()

In [6]:
actions

['drink water',
 'eat meal/snack',
 'brushing teeth',
 'brushing hair',
 'drop',
 'pickup',
 'throw',
 'sitting down',
 'standing up (from sitting position)',
 'clapping',
 'reading',
 'writing',
 'tear up paper',
 'wear jacket',
 'take off jacket',
 'wear a shoe',
 'take off a shoe',
 'wear on glasses',
 'take off glasses',
 'put on a hat/cap',
 'take off a hat/cap',
 'cheer up',
 'hand waving',
 'kicking something',
 'reach into pocket',
 'hopping (one foot jumping)',
 'jump up',
 'make a phone call/answer phone',
 'playing with phone/tablet',
 'typing on a keyboard',
 'pointing to something with finger',
 'taking a selfie',
 'check time (from watch)',
 'rub two hands together',
 'nod head/bow',
 'shake head',
 'wipe face',
 'salute',
 'put the palms together',
 'cross hands in front (say stop)',
 'sneeze/cough',
 'staggering',
 'falling',
 'touch head (headache)',
 'touch chest (stomachache/heart pain)',
 'touch back (backache)',
 'touch neck (neckache)',
 'nausea or vomiting condit

In [7]:
classes = [5, 6, 7, 8, 14, 24, 30, 32, 42]
for i,elem in enumerate(classes):
    print("class {} : {}".format(i, actions[elem]))

class 0 : pickup
class 1 : throw
class 2 : sitting down
class 3 : standing up (from sitting position)
class 4 : take off jacket
class 5 : reach into pocket
class 6 : pointing to something with finger
class 7 : check time (from watch)
class 8 : falling


In [8]:
class HumanActionDataset(Dataset):

    def __init__(self, data_dir, data_files, classes):
        self.data_dir = data_dir
        self.data_files = []
        for data_file in data_files:
            if int(data_file[17:-4])-1 in classes:
                self.data_files = self.data_files + [data_file]
        self.classes = classes

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        tensor = torch.Tensor(np.load(self.data_dir + self.data_files[idx]))
        tensor = tensor.reshape((tensor.shape[0], 75))
        label = int(self.data_files[idx][17:-4])-1
        label = self.classes.index(label)
        return (tensor, label)

In [9]:
HAD = HumanActionDataset(data_dir, data_files, classes)

___

## **2. Analysis**

### *2.1. General Overview*

In [10]:
general_info = [(i, HAD[i][0].shape[0], HAD[i][1]) for i in range(len(HAD))]
general_dataframe = pd.DataFrame(data=general_info, columns=["id", "nb_frame", "action_class"])

#### *2.1.A. Number of frames per represented classes*

For a given action class:
* ***count**: number of sample*
* ***mean**: mean number of frames*
* ***std**: standard deviation*
* ***min**: minimum number of frames*
* ***25%**: 1st quartile for the number of frames*
* ***50%**: median number of frames*
* ***75%**: 3rd quartile for the number of frames*
* ***max**: maximum number of frames*

In [14]:
for i in range(9):
    print("action: {}".format(actions[classes[i]]))
    display(general_dataframe[general_dataframe["action_class"] == i]["nb_frame"].describe())
    print("-"*30)

action: pickup


count    914.000000
mean      81.181619
std       13.367821
min       54.000000
25%       72.000000
50%       81.000000
75%       90.000000
max      131.000000
Name: nb_frame, dtype: float64

------------------------------
action: throw


count    914.000000
mean      63.861050
std       12.603714
min       37.000000
25%       55.000000
50%       62.000000
75%       71.000000
max      136.000000
Name: nb_frame, dtype: float64

------------------------------
action: sitting down


...

------------------------------
action: check time (from watch)


count    906.000000
mean      63.618102
std       14.124618
min       36.000000
25%       54.000000
50%       62.000000
75%       71.000000
max      128.000000
Name: nb_frame, dtype: float64

------------------------------
action: falling


count    890.000000
mean      63.823596
std       12.534735
min       40.000000
25%       56.000000
50%       62.000000
75%       71.000000
max      139.000000
Name: nb_frame, dtype: float64

------------------------------


#### *2.1.b. Joints*